In [1]:
import pandas as pd
import numpy as np

In [2]:
selloutData = pd.read_csv("../dataset/kopo_channel_seasonality_new.csv")
selloutData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY
0,A60,PRODUCT4,201402,71.0


In [3]:
selloutData["NEW_QTY"] = np.where(selloutData["QTY"]< 0, 0, selloutData["QTY"])

In [4]:
selloutData[selloutData.QTY >= 0].head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY
0,A60,PRODUCT4,201402,71.0,71.0
1,A60,PRODUCT59,201402,22275.0,22275.0
2,A60,PRODUCT34,201402,4463.0,4463.0
3,A60,PRODUCT47,201402,0.0,0.0
4,A60,PRODUCT56,201402,23.0,23.0


In [5]:
# 53주차 제거 2

In [6]:
selloutData.dtypes

REGIONID     object
PRODUCT      object
YEARWEEK      int64
QTY         float64
NEW_QTY     float64
dtype: object

In [7]:
## 연산과 자료형을 활용하여 데이터를 정제하기

In [8]:
def customFunction(inputColumn):
    yearValue = inputColumn / 100
    yearValue = int(yearValue)
    return yearValue

In [9]:
# 데이터를 정제하는 방법 활용 함수화, 연산, 자료형

In [10]:
# 방법 1 (자료형 변환)

In [11]:
selloutData["YEAR"] = (selloutData.YEARWEEK/100).astype(int)

In [12]:
# 방법 2 (함수 활용)

In [13]:
selloutData["YEAR"] = selloutData["YEARWEEK"].apply(customFunction)

In [14]:
# 방법 3 (연산자 활용)

In [15]:
selloutData["YEAR"] = selloutData["YEARWEEK"]//100

In [16]:
selloutData["WEEK"] = selloutData["YEARWEEK"]%100

In [17]:
refinedData = selloutData[selloutData.WEEK%100 < 53]

In [18]:
refinedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A60,PRODUCT4,201402,71.0,71.0,2014,2
1,A60,PRODUCT59,201402,22275.0,22275.0,2014,2
2,A60,PRODUCT34,201402,4463.0,4463.0,2014,2
3,A60,PRODUCT47,201402,0.0,0.0,2014,2
4,A60,PRODUCT56,201402,23.0,23.0,2014,2


In [19]:
sortkey = ['REGIONID', 'PRODUCT', 'YEARWEEK']

In [20]:
sortedData = refinedData.sort_values(sortkey)

In [21]:
sortedData.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
298,A00,PRODUCT34,201401,661.0,661.0,2014,1
1757,A00,PRODUCT34,201402,679.0,679.0,2014,2
3125,A00,PRODUCT34,201403,578.0,578.0,2014,3
205,A00,PRODUCT34,201404,532.0,532.0,2014,4
4369,A00,PRODUCT34,201405,516.0,516.0,2014,5


In [22]:
### 기존 인덱스를 제거하고 새로운 인덱스(정렬된)를 적용

In [23]:
step1Data = sortedData.reset_index(drop = True)

In [24]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK
0,A00,PRODUCT34,201401,661.0,661.0,2014,1
1,A00,PRODUCT34,201402,679.0,679.0,2014,2
2,A00,PRODUCT34,201403,578.0,578.0,2014,3
3,A00,PRODUCT34,201404,532.0,532.0,2014,4
4,A00,PRODUCT34,201405,516.0,516.0,2014,5


In [25]:
### 롤링함수를 적용가기전에는 정렬을 해줘야한다.

In [26]:
### 1. 기본 이동평균 함수

In [27]:
step1Data["MA"] = step1Data.NEW_QTY.rolling(window = 15, center=True, min_periods=1).mean()

In [28]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000


In [29]:
pwd

'E:\\GitMaster\\01.Python_Git\\04. Data Control'

In [30]:
# groupby 함수를 활용하여 원하는 데이터 정제

In [31]:
filteredData=["REGIONID","PRODUCT","YEAR"]

In [32]:
filteredData = step1Data[(step1Data.REGIONID=="A00")&(step1Data.PRODUCT=="PRODUCT34")]

In [34]:
filteredData.groupby(groupkey)[["NEW_QTY"]].mean()

NameError: name 'groupkey' is not defined

In [35]:
groupkey = ["REGIONID","PRODUCT","YEAR"]

In [36]:
groupData=step1Data.groupby(groupkey)[["NEW_QTY"]].mean()

In [37]:
groupData.columns = ["QTY_MEAN_YEAR"]

In [38]:
groupData.head()

QTY_MEAN_YEAR
REGIONID PRODUCT   YEAR               
A00      PRODUCT34 2014     275.961538
                   2015      86.634615
                   2016      36.576923
         PRODUCT58 2014       2.673077
                   2015       5.711538

In [39]:
groupData = groupData.reset_index()

In [40]:
groupData.head()

,REGIONID,PRODUCT,YEAR,QTY_MEAN_YEAR
0,A00,PRODUCT34,2014,275.961538
1,A00,PRODUCT34,2015,86.634615
2,A00,PRODUCT34,2016,36.576923
3,A00,PRODUCT58,2014,2.673077
4,A00,PRODUCT58,2015,5.711538


In [41]:
step1Data.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000
1,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444
2,A00,PRODUCT34,201403,578.0,578.0,2014,3,516.800000
3,A00,PRODUCT34,201404,532.0,532.0,2014,4,503.363636
4,A00,PRODUCT34,201405,516.0,516.0,2014,5,490.750000


In [42]:
joinKey = ["REGIONID","PRODUCT","YEAR"]

In [43]:
mergedData = pd.merge(groupData,step1Data, left_on=joinKey, right_on=joinKey)\
[["REGIONID","PRODUCT","YEARWEEK","YEAR","WEEK","QTY_MEAN_YEAR","NEW_QTY","MA"]]

In [44]:
mergedData.head()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_MEAN_YEAR,NEW_QTY,MA
0,A00,PRODUCT34,201401,2014,1,275.961538,661.0,520.000000
1,A00,PRODUCT34,201402,2014,2,275.961538,679.0,514.444444
2,A00,PRODUCT34,201403,2014,3,275.961538,578.0,516.800000
3,A00,PRODUCT34,201404,2014,4,275.961538,532.0,503.363636
4,A00,PRODUCT34,201405,2014,5,275.961538,516.0,490.750000


In [45]:
len(mergedData)

123864

In [46]:
leftData = step1Data

In [47]:
leftData.head(1)

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR,WEEK,MA
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.0


In [48]:
joinKey2=["REGIONID","PRODUCT"]

In [49]:
mergedData1 = pd.merge(leftData, groupData, how="left", on=joinKey2)

In [50]:
mergedData1.head()

,REGIONID,PRODUCT,YEARWEEK,QTY,NEW_QTY,YEAR_x,WEEK,MA,YEAR_y,QTY_MEAN_YEAR
0,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,2014,275.961538
1,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,2015,86.634615
2,A00,PRODUCT34,201401,661.0,661.0,2014,1,520.000000,2016,36.576923
3,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444,2014,275.961538
4,A00,PRODUCT34,201402,679.0,679.0,2014,2,514.444444,2015,86.634615


In [51]:
len(mergedData1)

371592

# 그룹바이함수 만들기

In [52]:
## 제품별로 그룹핑, 제품별 그룹핑을 하지않고 롤링을 하게 되면 원하지 않는 데이터간의 간섭이 일어나 
## 잘못된 데이터를 산출하게 됨
## 이를 막기위해 그룹바이 함수를 활용하여 지역,상품 별 그룹화를 한다

In [53]:
groupKey = ["REGIONID","PRODUCT"]

In [54]:
groupDataSet = mergedData.groupby(groupKey)

In [55]:
oneGroup = groupDataSet.get_group(list(groupDataSet.groups)[0])

In [56]:
oneGroup.head()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_MEAN_YEAR,NEW_QTY,MA
0,A00,PRODUCT34,201401,2014,1,275.961538,661.0,520.000000
1,A00,PRODUCT34,201402,2014,2,275.961538,679.0,514.444444
2,A00,PRODUCT34,201403,2014,3,275.961538,578.0,516.800000
3,A00,PRODUCT34,201404,2014,4,275.961538,532.0,503.363636
4,A00,PRODUCT34,201405,2014,5,275.961538,516.0,490.750000


In [57]:
def groupRolling(oneGroup):

    indexGroupData = oneGroup.reset_index(drop=True)
    indexGroupData["MA2"] = indexGroupData["NEW_QTY"].rolling(window=5, center=True).mean()
    
    return indexGroupData

In [58]:
finalResult = mergedData.groupby(groupKey).apply(groupRolling)

In [59]:
finalResult=finalResult.reset_index(drop=True)

In [60]:
finalResult.head()

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_MEAN_YEAR,NEW_QTY,MA,MA2
0,A00,PRODUCT34,201401,2014,1,275.961538,661.0,520.000000,NaN
1,A00,PRODUCT34,201402,2014,2,275.961538,679.0,514.444444,NaN
2,A00,PRODUCT34,201403,2014,3,275.961538,578.0,516.800000,593.2
3,A00,PRODUCT34,201404,2014,4,275.961538,532.0,503.363636,545.6
4,A00,PRODUCT34,201405,2014,5,275.961538,516.0,490.750000,491.2


In [61]:
finalResult.to_csv("./finalResult3.csv")

In [62]:
fullData = selloutData

In [63]:
a01Data = fullData[fullData.REGIONID=="A01"]
len(a01Data)

3611

In [64]:
a02Data = fullData[fullData.REGIONID=="A20"]
len(a02Data)

471

In [65]:
resultConcat = pd.concat([a01Data,a02Data], axis=0)

In [66]:
len(resultConcat)

4082

In [67]:
## 특정 칼럼 이름 변경

In [68]:
finalResult = finalResult.rename(columns = {'MEAN_YEAR': 'QTY_MEAN_YEAR','WEEK1':'WEEK'})

In [69]:
## 특정 칼럼 삭제

In [70]:
# finalResult 데이터의 상품별 QTY의 마지막 4주차동안의 평균값을 컬럼에 추가

In [71]:
answer1 = finalResult["YEARWEEK"].max()

In [72]:
answer1

201652

In [73]:
reData1 = finalResult[(finalResult.YEARWEEK<=answer1) & (finalResult.YEARWEEK>answer1-4)]

In [74]:
reData1

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_MEAN_YEAR,NEW_QTY,MA,MA2
152,A00,PRODUCT34,201649,2016,49,36.576923,19.0,16.200000,30.6
153,A00,PRODUCT34,201650,2016,50,36.576923,35.0,15.533333,33.2
154,A00,PRODUCT34,201651,2016,51,36.576923,73.0,14.800000,NaN
155,A00,PRODUCT34,201652,2016,52,36.576923,22.0,13.800000,NaN
308,A00,PRODUCT58,201649,2016,49,307.365385,155.0,2488.000000,120.4
309,A00,PRODUCT58,201650,2016,50,307.365385,104.0,3168.866667,96.0
310,A00,PRODUCT58,201651,2016,51,307.365385,27.0,4078.466667,NaN
311,A00,PRODUCT58,201652,2016,52,307.365385,19.0,5217.066667,NaN
464,A00,PRODUCT59,201649,2016,49,28213.173077,22993.0,19892.933333,23588.8
465,A00,PRODUCT59,201650,2016,50,28213.173077,26854.0,19360.400000,26344.4


In [75]:
groupkey = ["REGIONID","PRODUCT"]

In [76]:
eachGroupData = reData1.groupby(groupkey)[["NEW_QTY"]].mean()

In [77]:
eachGroupData.reset_index().head()

,REGIONID,PRODUCT,NEW_QTY
0,A00,PRODUCT34,37.25
1,A00,PRODUCT58,76.25
2,A00,PRODUCT59,28398.75
3,A01,PRODUCT1,390.75
4,A01,PRODUCT12,302132.00


In [78]:
realFinalData = pd.merge(finalResult, eachGroupData, how="left", on=groupkey)

In [79]:
realFinalData = realFinalData.rename(columns = {'NEW_QTY_y': 'MEAN'})

In [80]:
realFinalData

,REGIONID,PRODUCT,YEARWEEK,YEAR,WEEK,QTY_MEAN_YEAR,NEW_QTY_x,MA,MA2,MEAN
0,A00,PRODUCT34,201401,2014,1,275.961538,661.0,520.000000,NaN,37.25
1,A00,PRODUCT34,201402,2014,2,275.961538,679.0,514.444444,NaN,37.25
2,A00,PRODUCT34,201403,2014,3,275.961538,578.0,516.800000,593.2,37.25
3,A00,PRODUCT34,201404,2014,4,275.961538,532.0,503.363636,545.6,37.25
4,A00,PRODUCT34,201405,2014,5,275.961538,516.0,490.750000,491.2,37.25
5,A00,PRODUCT34,201406,2014,6,275.961538,423.0,480.230769,448.4,37.25
6,A00,PRODUCT34,201407,2014,7,275.961538,407.0,468.214286,436.0,37.25
7,A00,PRODUCT34,201408,2014,8,275.961538,364.0,456.666667,440.4,37.25
8,A00,PRODUCT34,201409,2014,9,275.961538,470.0,431.533333,429.6,37.25
9,A00,PRODUCT34,201410,2014,10,275.961538,538.0,402.066667,418.6,37.25
